# Lab-4

In [ ]:
# 1. Генерація зображень
# Вирішіть завдання генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE), якщо в обраному датасеті багато класів, залиште декілька.
# Датасети: можна брати CIFAR-100, Fashion MNIST або тут: https://www.kaggle.com/tags/image-data  

# 2.  Вирішити завдання машинного перекладу (будь яка пара мов, датасети тут: https://www.manythings.org/anki/)
# Приклад: https://keras.io/examples/nlp/neural_machine_translation_with_transformer/ (English-to-Spanish не обирати!)

# 3. Проведіть експерименти з моделями бібліотеки HF Transformers (https://huggingface.co/) за допомогою (наприклад) Pipeline або TFAutoModelForSequenceClassification
# Приклад: https://github.com/natsakh/Data-Analysis/blob/main/Pr_8/1%20hugging%20face%20transformers.ipynb

# 4. Бонус (не обов'язкова задача). Застосуйте pretrained BERT model для задачі класифікації текстів з лабораторної 2. 
# Приклад: https://github.com/natsakh/Data-Analysis/blob/main/Pr_8/2%20hugging%20face%20transformers.ipynb

# Як звіт – робочий код в Jupyter notebook заливаєте на свій репозиторій. Лінк відправляєте як відповідь до цього завдання.
# Максимальний бал – 10, 6 за роботу + 4 захист.
# Захистити роботу можна буде в грудні на практиці (якщо встигнете виконати) або на консультації перед екзаменом.
# (!) Захищати на консультації перед екзаменом можна буде тільки лаб 4, лаб 1, 2, 3 - тільки в семестрі, на практичних
# (!) Однакові або дуже схожі роботи прийматися не будуть.

# Image generation

# Machine translation

# HF Transformers 

# Sources